In [6]:
import pandas as pd
from Bio import SeqIO
import gzip
import re

Необходимо иметь файл с прочтениями и файл с таблицей таксономического профилирования Kraken2

In [11]:
# файл с прочтениями, которые не были картированы на геном хозяина (в данном случае - на геном летучих мышей)
file_path_fastq = "D:\\host_predicition\\raw_data\\SAMN37347010_merged_unmapped.fastq.gz"

# файл с таблицей - вывод профилирования программой Kraken2
file_path_kraken2 = "D:\\host_predicition\\raw_data\\SAMN37347010_k2.output"

In [99]:
df = pd.read_csv(file_path_kraken2, sep="\t")

Отберём все таксоны, содержащие слово "virus"

In [ ]:
virus_df = df[df.taxa_id.str.contains("virus")]

Выделим "virus_name" и "taxid" в отдельные столбцы

In [121]:
virus_df["virus_name"], virus_df["taxid"] = virus_df.taxa_id.str.split("(", n=1).str
virus_df.taxid = virus_df.taxid.str[6:-1]

C:\Users\Фёдор\AppData\Local\Temp\ipykernel_12932\16453385.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  virus_df["virus_name"], virus_df["taxid"] = virus_df.taxa_id.str.split("(", n=1).str
C:\Users\Фёдор\AppData\Local\Temp\ipykernel_12932\16453385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virus_df["virus_name"], virus_df["taxid"] = virus_df.taxa_id.str.split("(", n=1).str
C:\Users\Фёдор\AppData\Local\Temp\ipykernel_12932\16453385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [122]:
virus_df.head()

,c/u,sequence_id,taxa_id,read_length,map,virus_name,taxid
13,C,SAMN37347010.55,Parus major densovirus (taxid 1907771),178,1907771:144,Parus major densovirus,1907771
161,C,SAMN37347010.469,Parus major densovirus (taxid 1907771),191,0:76 1907771:12 0:69,Parus major densovirus,1907771
226,C,SAMN37347010.553,Parus major densovirus (taxid 1907771),161,1907771:56 0:9 1907771:5 0:5 1907771:3 0:10 19...,Parus major densovirus,1907771
260,C,SAMN37347010.666,Parus major densovirus (taxid 1907771),164,0:11 1907771:1 0:5 1907771:3 0:5 1907771:105,Parus major densovirus,1907771
286,C,SAMN37347010.460,Parus major densovirus (taxid 1907771),204,0:7 1907771:52 0:10 1907771:1 0:12 1907771:1 0...,Parus major densovirus,1907771


Выделим уникальные таксоны вирусов, запишем переменную "unique_viruses"

In [3]:
unique_viruses = virus_df["virus_name"].unique()

NameError: name 'virus_df' is not defined

Оценим число вирусных прочтений в образце, а также количество уникальных таксонов вирусов

In [18]:
print("Число вирусных прочтений")
len(virus_df["virus_name"])

Число вирусных прочтений


405460

In [19]:
print("Уникальные таксоны вирусов")
len(unique_viruses)

Уникальные таксоны вирусов


83

Сделаем для себя таблицу "raw_data_annotation.csv", содержащую столбцы "virus_name" и "taxid".

In [5]:
name, taxid = [], []
for el in unique_viruses:
    tmp = el.split("(")
    name.append(tmp[0][:-1])
    taxid.append(int(tmp[1][6:-1]))
    
out = pd.DataFrame(columns=["virus_name", "taxid"])
out.virus_name, out.taxid = name, taxid

out.to_csv("D:\\host_predicition\\raw_data\\raw_data_annotation.csv", sep=",")

NameError: name 'unique_viruses' is not defined

Далее таблица "raw_data_annotation.csv" аннотируется вручную - необходимо получить информацию о семействах вирусов, типе нуклеиновой кислоты (ss/ds;RNA(+/-)/DNA) и группе хозяев, которых они заражают.

# Annotated data

На этом этапе работаем с проаннотированной таблицей "raw_data_annotation.csv"
(в открытом доступе https://docs.google.com/spreadsheets/d/1OB3UgJP7BvJG4K9QjNpSYP3HikrTcYF2qJU5odcgGYg/edit?usp=sharing)

taxid РНК-вирусов выделяется в переменную ids.

In [115]:
df_annotation = pd.read_csv("D:\\host_predicition\\raw_data\\raw_data_annotation.csv", sep=",")
ids = df_annotation[df_annotation.nucleic_acid.str.contains("RNA", na=False)].taxid.astype("str").values

Отбираем все индексы прочтений по колонке taxid, которые принадлежат РНК-вирусам

Для данного метагеномного сэмпла (SAMN37347010) получилось так, что все РНК-вирусы заражают лишь исследуемые группы хозяев - млекопитающих, растения и насекомых. Поэтому фильтра по хозяевам нет, можно добавить в будущем, если понадобится.
Тем более, в результате собрались контиги всего лишь трёх РНК-вирусов, которые 100% заражают лишь млеков и растения

In [130]:
rna_virus_reads_ids = virus_df.set_index("taxid", drop=True).loc[ids].sequence_id.values

В переменной "rna_virus_reads_ids" содержатся индексы прочтений РНК-вирусов, заражающих млекопитающих, растения и насекомых

In [131]:
rna_virus_reads_ids

array(['SAMN37347010.1530', 'SAMN37347010.2261', 'SAMN37347010.4134', ...,
       'SAMN37347010.26400958', 'SAMN37347010.34260649',
       'SAMN37347010.30561442'], dtype=object)

In [128]:
print("Число прочтений РНК-вирусов, заражающих млекопитающих, растения или насекомых:", len(rna_virus_reads_ids))

Число прочтений РНК-вирусов, заражающих млекопитающих, растения или насекомых: 11925


# Parse fastq.gz file to extract RNA viral reads

Парсим файл с прочтениями, сохраняем лишь по индексу из переменной rna_virus_reads_ids

In [132]:
with open("D:\\host_predicition\\raw_data\\rna_viral_reads.fastq", "w") as output_file:
    for seq in SeqIO.parse(gzip.open(file_path_fastq, "rt"), "fastq"):
        if seq.id in rna_virus_reads_ids:
            SeqIO.write(seq, output_file, "fastq")

# Trash

In [73]:
regex = (r"(?P<virus_name>.*\()")
virus_df["virus_name"] = virus_df.taxa_id.str.extract(regex).virus_name.str[:-2]

regex = (r"(?P<taxid>[0-9]+)")
virus_df["taxid"] = virus_df.taxa_id.str.extract(regex).taxid

C:\Users\Фёдор\AppData\Local\Temp\ipykernel_12932\2989797323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virus_df["virus_name"] = virus_df.taxa_id.str.extract(regex).virus_name.str[:-2]
C:\Users\Фёдор\AppData\Local\Temp\ipykernel_12932\2989797323.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virus_df["taxid"] = virus_df.taxa_id.str.extract(regex).taxid
